## 161: Verificación de correo electrónico al registrarse (con tokens temporales)

Implementar un sistema que permita verificar el correo electrónico de un usuario recién registrado. Esto se logra mediante el envío (simulado) de un token temporal de activación que debe confirmar.

🧩 Estructura del flujo
El usuario se registra con su correo.

Se genera un token JWT temporal de activación.

El sistema simula el envío del token por email.

El usuario accede a una ruta para confirmar su cuenta con el token.

Su cuenta se marca como verificada.

✅ Código completo
python
Copiar
Editar
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel, EmailStr
from sqlalchemy.orm import Session
from datetime import datetime, timedelta
from jose import jwt, JWTError
from passlib.context import CryptContext
from sqlalchemy import Column, Integer, String, Boolean, create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

# --- Configuración ---
app = FastAPI()
DATABASE_URL = "sqlite:///./usuarios.db"
SECRET_KEY = "CLAVE_VERIFICACION"
ALGORITHM = "HS256"
ACTIVATION_TOKEN_EXPIRE_MINUTES = 30

Base = declarative_base()
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(bind=engine, autoflush=False)
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# --- Modelo SQL ---
class Usuario(Base):
    __tablename__ = "usuarios"
    id = Column(Integer, primary_key=True, index=True)
    email = Column(String, unique=True, index=True)
    contraseña = Column(String)
    verificado = Column(Boolean, default=False)

Base.metadata.create_all(bind=engine)

# --- Modelos Pydantic ---
class RegistroUsuario(BaseModel):
    email: EmailStr
    contraseña: str

class ConfirmacionToken(BaseModel):
    token: str

# --- Utilidades ---
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

def hashear_contraseña(contraseña: str) -> str:
    return pwd_context.hash(contraseña)

# --- 1. Registro con token de activación ---
@app.post("/registro")
def registrar(usuario: RegistroUsuario, db: Session = Depends(get_db)):
    existente = db.query(Usuario).filter(Usuario.email == usuario.email).first()
    if existente:
        raise HTTPException(status_code=400, detail="Email ya registrado")

    nuevo_usuario = Usuario(
        email=usuario.email,
        contraseña=hashear_contraseña(usuario.contraseña),
        verificado=False
    )
    db.add(nuevo_usuario)
    db.commit()

    # Generar token de activación
    expiracion = datetime.utcnow() + timedelta(minutes=ACTIVATION_TOKEN_EXPIRE_MINUTES)
    token_data = {"sub": usuario.email, "exp": expiracion}
    token = jwt.encode(token_data, SECRET_KEY, algorithm=ALGORITHM)

    # Simulación de envío de correo
    return {
        "mensaje": "Cuenta registrada. Verifica tu correo para activar la cuenta.",
        "enlace_de_activacion": f"http://localhost:8000/activar?token={token}"
    }

# --- 2. Confirmar cuenta con token ---
@app.post("/activar")
def activar(data: ConfirmacionToken, db: Session = Depends(get_db)):
    try:
        payload = jwt.decode(data.token, SECRET_KEY, algorithms=[ALGORITHM])
        email = payload.get("sub")
        if not email:
            raise HTTPException(status_code=400, detail="Token inválido")
    except JWTError:
        raise HTTPException(status_code=400, detail="Token inválido o expirado")

    usuario = db.query(Usuario).filter(Usuario.email == email).first()
    if not usuario:
        raise HTTPException(status_code=404, detail="Usuario no encontrado")

    if usuario.verificado:
        return {"mensaje": "Cuenta ya está verificada"}

    usuario.verificado = True
    db.commit()
    return {"mensaje": "Cuenta verificada exitosamente"}